## use it in PyTorch

In [1]:
data_dir = "data"

from dataset import syntax_token_type
syntax_vocabulary, blacklist = syntax_token_type(data_dir)

encountering ambiguous tag at data/sw2260#s283_500/syntax.txt, break-tied
encountering type II typo at data/sw2249#s67_500/syntax.txt, blacklisted
encountering type II typo at data/sw2789#s226_500/syntax.txt, blacklisted
encountering type II typo at data/sw2229#s140_500/syntax.txt, blacklisted
encountering type II typo at data/sw2589#s154_500/syntax.txt, blacklisted
encountering type II typo at data/sw2067#s50_500/syntax.txt, blacklisted
encountering type II typo at data/sw2154#s6_500/syntax.txt, blacklisted
encountering ambiguous tag at data/sw2015#s53_500/syntax.txt, break-tied
encountering type II typo at data/sw2079#s10_500/syntax.txt, blacklisted
encountering type II typo at data/sw2434#s87_500/syntax.txt, blacklisted
encountering a weird semicolon at data/sw3049#s167_500/syntax.txt, blacklisted
encountering type II typo at data/sw2229#s232_500/syntax.txt, blacklisted
encountering type II typo at data/sw2249#s194_500/syntax.txt, blacklisted
encountering type II typo at data/sw2641

Syntax Token Count: 
defaultdict(<class 'int'>, {'S': 87661, 'NP': 176702, 'PRP': 66885, '': 1114267, 'VP': 117204, 'VBP': 33049, 'PUNC': 23165, '[': 42621, ']': 42621, 'PRN': 6986, 'SIL': 68195, 'VBN': 6294, 'DT': 40144, 'JJ': 20737, 'NN': 43006, 'PP': 32323, 'IN': 40646, 'JJR': 1193, 'ADVP': 27792, 'RB': 42545, 'NNP': 7024, 'POS': 427, 'EDITED': 15382, 'PRP$': 5126, 'NNS': 14659, 'VB': 20418, 'VBD': 15575, 'VBG': 7370, 'CC': 23598, 'SBAR': 22398, 'VBZ': 7487, 'ADJP': 11371, 'MD': 6431, 'TRACE': 32068, 'TO': 8211, 'BES': 9112, 'WHNP': 6487, 'WHADJP': 124, 'WRB': 3103, 'WHADVP': 3694, 'SBARQ': 169, 'SQ': 262, 'INTJ': 16617, 'UH': 16360, 'WP': 2194, 'X': 624, 'UCP': 263, 'CD': 5255, 'PRT': 2438, 'RP': 2552, 'WDT': 3279, 'CONJP': 1301, 'QP': 1080, 'TYPO': 525, 'EX': 1538, 'RBR': 670, 'FRAG': 285, 'RBS': 106, 'NNPS': 432, 'XX': 553, 'PDT': 815, 'HVS': 571, 'GW': 234, 'NAC': 261, 'JJS': 709, 'UNK': 142, 'NX': 255, 'FW': 68, 'SYM': 52, 'SINV': 21, 'WHPP': 14, 'RRC': 11, 'WP$': 6, 'LST': 8, 

In [2]:
print(syntax_vocabulary)
print(blacklist)

{'': 0, 'ADJP': 1, 'ADVP': 2, 'BES': 3, 'CC': 4, 'CD': 5, 'CONJP': 6, 'DT': 7, 'EDITED': 8, 'EX': 9, 'FRAG': 10, 'FW': 11, 'GW': 12, 'HVS': 13, 'IN': 14, 'INTJ': 15, 'JJ': 16, 'JJR': 17, 'JJS': 18, 'LS': 19, 'LST': 20, 'MD': 21, 'NAC': 22, 'NN': 23, 'NNP': 24, 'NNPS': 25, 'NNS': 26, 'NP': 27, 'NX': 28, 'PDT': 29, 'POS': 30, 'PP': 31, 'PRN': 32, 'PRP': 33, 'PRP$': 34, 'PRT': 35, 'PUNC': 36, 'QP': 37, 'RB': 38, 'RBR': 39, 'RBS': 40, 'RP': 41, 'RRC': 42, 'S': 43, 'SBAR': 44, 'SBARQ': 45, 'SIL': 46, 'SINV': 47, 'SQ': 48, 'SYM': 49, 'TO': 50, 'TRACE': 51, 'TYPO': 52, 'UCP': 53, 'UH': 54, 'UNK': 55, 'VB': 56, 'VBD': 57, 'VBG': 58, 'VBN': 59, 'VBP': 60, 'VBZ': 61, 'VP': 62, 'WDT': 63, 'WHADJP': 64, 'WHADVP': 65, 'WHNP': 66, 'WHPP': 67, 'WP': 68, 'WP$': 69, 'WRB': 70, 'X': 71, 'XX': 72, '[': 73, ']': 74}
{'data/sw2641#s78_500', 'data/sw2589#s154_500', 'data/sw2249#s67_500', 'data/sw2249#s194_500', 'data/sw2229#s232_500', 'data/sw2067#s50_500', 'data/sw2154#s6_500', 'data/sw2249#s149_500', 'dat

In [3]:
from dataset import SpeechSyntax, Phase, get_dataloader

dataloader = get_dataloader(
    root="data", phase=Phase.TRAIN, syntax_vocabulary=syntax_vocabulary, blacklist=blacklist,
    batch_size=4, shuffle=True, num_workers=2
)

This is only for demo. You probably will never call ```pad_packed_sequence``` explicitly, which wastes a good deal of memory

In [24]:
from torch.nn.utils.rnn import PackedSequence, pad_packed_sequence

for speech, syntax in dataloader:
    assert type(speech) == PackedSequence
    assert type(syntax) == PackedSequence
    
    # they can already be fed into a RNN
    
    unpacked_speech, unpacked_speech_len = pad_packed_sequence(speech, batch_first=True)
    print("speech batch")
    print(unpacked_speech.shape)
    print(unpacked_speech_len)
    
    unpacked_syntax, unpacked_syntax_len = pad_packed_sequence(syntax, batch_first=True)
    print("syntax batch")
    print(unpacked_syntax.shape)
    print(unpacked_syntax_len)
    
    print("It's only a demo, people.")
    break

speech batch
torch.Size([4, 25085])
tensor([25085, 21939, 13864, 10477])
syntax batch
torch.Size([4, 72])
tensor([72, 54, 44, 40])
It's only a demo, people.


In [25]:
import IPython.display as ipd

def display_speech(unpacked_speech, unpacked_speech_len):
    auds = []
    for s, l in zip(unpacked_speech, unpacked_speech_len):
        speech = s[:l].numpy()
        auds.append(ipd.Audio(speech, rate=8000))
    return auds
        

In [26]:
auds = display_speech(unpacked_speech, unpacked_speech_len)

In [27]:
auds[0]

In [28]:
auds[1]

In [29]:
auds[2]

In [30]:
auds[3]